# KNN using Poincare Ball

In [1]:
from hyptorch.nn import ToPoincare
from hyptorch.pmath import dist as hypdist

import numpy as np  # NumPy for numerical operations
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.init as init

from collections import Counter  # Counter for counting votes in the majority voting process
import matplotlib.pyplot as plt  # Matplotlib for data visualization
from matplotlib.colors import ListedColormap  # ListedColormap for custom color maps
from sklearn import datasets  # Scikit-Learn for dataset loading
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.metrics import accuracy_score, balanced_accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler

from sklearn.neighbors import KNeighborsClassifier

In [2]:
# TODO-LIST:
# 1- Converter os dados para a bola de poincare; Check ✓
# 2- Calcular a distancia na bola de poincare; Check ✓
# 3- Validar se esta na bola de poincare;
# 4- Plotar o CO-TSNE

In [76]:
# Original Code: https://github.com/mouraffa/KNN-From-Scratch-Iris-Classifier

# ---------------------------------------------------------------------------
# Implementação do KNN Euclidiano
# ---------------------------------------------------------------------------

def euclidean_distance(x1, x2):
    """
    Calculate the Euclidean distance between two data points x1 and x2.

    Parameters:
    x1 (numpy.ndarray): The first data point.
    x2 (numpy.ndarray): The second data point.

    Returns:
    float: The Euclidean distance between x1 and x2.
    """
    distance = np.sqrt(np.sum((x1 - x2) ** 2))
    return distance

class KNN:
    def __init__(self, k: int = 3):
        """
        Inicializa o classificador KNN.

        Parâmetros:
        k (int): O número de vizinhos a considerar (padrão é 3).
        """
        if k <= 0:
            raise ValueError("O número de vizinhos (k) deve ser positivo.")
        
        self.k = k
        self.X_train = None
        self.y_train = None

    def fit(self, X: np.ndarray, y: np.ndarray) -> None:
        """
        Treina o classificador KNN com os dados de treinamento.

        Parâmetros:
        X (numpy.ndarray): As características dos dados de treinamento.
        y (numpy.ndarray): Os rótulos dos dados de treinamento.
        """
        if X.shape[0] != y.shape[0]:
            raise ValueError("O número de amostras em X e y deve ser o mesmo.")
        if X.shape[0] < self.k:
            # Aviso ou erro, pois não haverá k vizinhos suficientes
            print(f"Aviso: O número de amostras de treinamento ({X.shape[0]}) é menor que k ({self.k}).")

        self.X_train = X
        self.y_train = y

    def predict(self, X_test: np.ndarray) -> np.ndarray:
        """
        Prevê os rótulos para um conjunto de pontos de dados X_test.

        Parâmetros:
        X_test (numpy.ndarray): Os pontos de dados para os quais fazer previsões.

        Retorna:
        numpy.ndarray: Um array de rótulos previstos.
        """
        if self.X_train is None or self.y_train is None:
            raise RuntimeError("O classificador KNN deve ser treinado com o método 'fit' antes de prever.")
        
        predictions = [self._predict_single(x) for x in X_test]
        return np.array(predictions)
    
    def _predict_single(self, x: np.ndarray) -> int: # Renomeado para clareza e consistência
        """
        Prevê o rótulo para um único ponto de dados x.

        Parâmetros:
        x (numpy.ndarray): O ponto de dados para o qual fazer uma previsão.

        Retorna:
        int: O rótulo previsto para o ponto de dados de entrada x.
        """
        # Calcula as distâncias para todos os pontos de dados de treinamento
        distances = [euclidean_distance(x, x_train_point) for x_train_point in self.X_train]

        # Obtém os índices dos k pontos de dados de treinamento mais próximos
        k_indices = np.argsort(distances)[:self.k]

        # Obtém os rótulos dos k pontos de dados de treinamento mais próximos
        k_nearest_labels = [self.y_train[i] for i in k_indices]

        # Realiza a votação majoritária para determinar o rótulo final
        # Counter(k_nearest_labels) cria um dict-like: {label: count}
        # .most_common(1) retorna uma lista com uma tupla: [(most_common_label, count)]
        most_common = Counter(k_nearest_labels).most_common(1)
        return most_common[0][0]

# ---------------------------------------------------------------------------
# Implementação do KNN Hiperbólico (Modelo de Disco de Poincaré)
# ---------------------------------------------------------------------------

class HyperbolicKNN:
    def __init__(self, k: int = 3, c: float = 0.5, validate_input_geometry: bool = True):
        """
        Inicializa o classificador KNN Hiperbólico.

        Parâmetros:
        k (int): O número de vizinhos a considerar (padrão é 3).
        c (float): Parâmetro de curvatura da bola de Poincaré (geralmente $c > 0$, onde
                   a curvatura do espaço $K = -c$ ou $K = -c^2$ dependendo da convenção.
                   Se o raio da bola é $1/\\sqrt{c_K}$ para curvatura $K > 0$, então
                   esta `c` corresponde a $c_K$. Se o raio é $1/c_R$, então esta `c` é $c_R^2$.
                   A validação usa $1/\\sqrt{c}$, então `c` é assumida como $K > 0$.
        validate_input_geometry (bool): Se True, valida se os pontos estão na bola de Poincaré.
        """
        if k <= 0:
            raise ValueError("O número de vizinhos (k) deve ser positivo.")
        if c <= 0:
            raise ValueError("O parâmetro de curvatura 'c' deve ser positivo para o modelo de Poincaré conforme usado na validação.")
        self.k = k
        self.c = c
        self.validate_input_geometry = validate_input_geometry
        self.X_train = None
        self.y_train = None

    def _validate_poincare_points(self, X: np.ndarray) -> None:
        """
        Valida se os pontos de entrada estão dentro da bola de Poincaré.

        Args:
            X (np.ndarray): Array de pontos, onde cada linha é um ponto.
            curvature (float): A curvatura do espaço hiperbólico. Deve ser positiva.

        Raises:
            ValueError: Se a curvatura não for positiva.
            AssertionError: Se algum ponto estiver fora da bola de Poincaré
                            definida por 1 / sqrt(curvature).
        """
        if self.c <= 0:
            raise ValueError("A curvatura (K) deve ser um valor positivo.")
        
        max_norm_allowed = 1 / (self.c**0.5)
        norms = np.linalg.norm(X, axis=1)

        if not np.all(norms <= max_norm_allowed):
            problematic_indices = np.where(norms > max_norm_allowed)[0]
            error_msg = (
                f"Validação da geometria falhou! Pontos com índices {problematic_indices} "
                f"estão fora da bola de Poincaré (norma > {max_norm_allowed:.4f} para K={self.c}).\n"
                f"Normas encontradas para esses pontos: {norms[problematic_indices]}"
            )
            raise AssertionError(error_msg)
        # print(f"Validação da geometria de Poincaré para {X.shape[0]} pontos bem-sucedida (K={curvature}).")
        
    def fit(self, X: np.ndarray, y: np.ndarray) -> None:
        """
        Treina o classificador KNN Hiperbólico com os dados de treinamento.

        Parâmetros:
        X (numpy.ndarray): As características dos dados de treinamento (pontos na bola de Poincaré).
        y (numpy.ndarray): Os rótulos dos dados de treinamento.
        """
        if X.shape[0] != y.shape[0]:
            raise ValueError("O número de amostras em X e y deve ser o mesmo.")
        if X.shape[0] < self.k:
             print(f"Aviso: O número de amostras de treinamento ({X.shape[0]}) é menor que k ({self.k}).")

        if self.validate_input_geometry:
            self._validate_poincare_points(X)

        self.X_train = X
        self.y_train = y

    def predict(self, X_test: np.ndarray) -> np.ndarray:
        """
        Prevê os rótulos para um conjunto de pontos de dados X_test.

        Parâmetros:
        X_test (numpy.ndarray): Os pontos de dados (na bola de Poincaré) para os quais fazer previsões.

        Retorna:
        numpy.ndarray: Um array de rótulos previstos.
        """
        if self.X_train is None or self.y_train is None:
            raise RuntimeError("O classificador KNN deve ser treinado com o método 'fit' antes de prever.")

        if self.validate_input_geometry:
            self._validate_poincare_points(X_test)
        
        predictions = [self._predict_single(x) for x in X_test]
        return np.array(predictions)



    def _predict_single(self, x: np.ndarray) -> int: # Renomeado para clareza e consistência
        """
        Prevê o rótulo para um único ponto de dados x na bola de Poincaré.

        Parâmetros:
        x (numpy.ndarray): O ponto de dados para o qual fazer uma previsão.

        Retorna:
        int: O rótulo previsto para o ponto de dados de entrada x.
        """
        # Calcula as distâncias hiperbólicas para todos os pontos de dados de treinamento
        # A função `hypdist` DEVE ser uma implementação correta da distância no modelo de Poincaré.
        distances = [hypdist(x, x_train_point, c=self.c) for x_train_point in self.X_train]

        # Obtém os índices dos k pontos de dados de treinamento mais próximos
        k_indices = np.argsort(distances)[:self.k]

        # Obtém os rótulos dos k pontos de dados de treinamento mais próximos
        k_nearest_labels = [self.y_train[i] for i in k_indices]

        # Realiza a votação majoritária para determinar o rótulo final
        most_common = Counter(k_nearest_labels).most_common(1)
        return most_common[0][0]


# Iris Dataset

In [83]:
# --- Configurações e Constantes ---
SEED = 78645
VALIDATE_INPUT_GEOMETRY = True
CURVATURE = 0.1  # Curvatura para o espaço de Poincaré
N_NEIGHBORS = 3  # Número de vizinhos para o KNN
TEST_SIZE_INITIAL_SPLIT = 0.1  # Proporção para o conjunto de teste
TEST_SIZE_VALIDATION_SPLIT = 0.1 # Proporção para o conjunto de validação (do restante)


# Define a semente para reprodutibilidade
np.random.seed(SEED)
torch.manual_seed(SEED)

# --- 1. Carregamento e Divisão dos Dados ---
print("Carregando dataset Iris...")
iris = datasets.load_iris()
data_np, data_labels_np = iris.data, iris.target

# Divide o dataset em treino+validação e teste
x_train_val_np, X_test_np, y_train_val_np, y_test_np = train_test_split(
    data_np,
    data_labels_np,
    stratify=data_labels_np,
    test_size=TEST_SIZE_INITIAL_SPLIT,
    random_state=SEED,
)

# Divide o conjunto treino+validação em treino e validação
X_train_np, X_valid_np, y_train_np, y_valid_np = train_test_split(
    x_train_val_np,
    y_train_val_np,
    stratify=y_train_val_np,
    test_size=TEST_SIZE_VALIDATION_SPLIT,
    random_state=SEED,
)

# --- 2. Conversão para Tensores PyTorch ---
X_train = torch.Tensor(X_train_np)
y_train = torch.Tensor(y_train_np).long()  # Rótulos como LongTensor
X_valid = torch.Tensor(X_valid_np)
y_valid = torch.Tensor(y_valid_np).long()  # Rótulos como LongTensor
X_test = torch.Tensor(X_test_np)
y_test = torch.Tensor(y_test_np).long()    # Rótulos como LongTensor

# --- 3. Transformação para o Espaço de Poincaré e Treinamento do Modelo ---
print(f"\nIniciando modelagem em Espaço Hiperbólico (Curvatura K={CURVATURE})...")

# Inicializa o transformador Euclidiano para Poincaré
e2p_transformer = ToPoincare(c=CURVATURE, train_c=False, train_x=False)

# Mapeia os dados para a bola de Poincaré
X_train_hyp = e2p_transformer(X_train)
X_valid_hyp = e2p_transformer(X_valid)
X_test_hyp = e2p_transformer(X_test)

# Inicializa e treina o classificador KNN Hiperbólico
hyperbolic_knn_classifier = HyperbolicKNN(k=N_NEIGHBORS, c=CURVATURE)
hyperbolic_knn_classifier.fit(X_train_hyp, y_train)

# Faz predições no conjunto de teste
y_pred_eval = hyperbolic_knn_classifier.predict(X_test_hyp)

# --- 4. Avaliação do Modelo ---

# Converte tensores para arrays NumPy para uso com sklearn.metrics
y_test_eval = y_test.cpu().numpy()

accuracy = accuracy_score(y_test_eval, y_pred_eval)
balanced_accuracy = balanced_accuracy_score(y_test_eval, y_pred_eval)
# average='weighted' para lidar com desbalanceamento de classes nas métricas de recall, precision, f1
recall = recall_score(y_test_eval, y_pred_eval, average='weighted', zero_division=0)
precision = precision_score(y_test_eval, y_pred_eval, average='weighted', zero_division=0)
f1 = f1_score(y_test_eval, y_pred_eval, average='weighted', zero_division=0)

print("\n--- Resultados da Avaliação ---")
print(f"Acurácia: {accuracy:.4f}")
print(f"Acurácia Balanceada: {balanced_accuracy:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Precisão: {precision:.4f}")
print(f"F1-Score: {f1:.4f}")


Carregando dataset Iris...

Iniciando modelagem em Espaço Hiperbólico (Curvatura K=0.1)...

--- Resultados da Avaliação ---
Acurácia: 1.0000
Acurácia Balanceada: 1.0000
Recall: 1.0000
Precisão: 1.0000
F1-Score: 1.0000


# Pad-Ufes-20

In [89]:
# #############################
# # Initialize and train the KNN classifier
# knn_classifier = KNN(k=2)  # Create a KNN classifier with k=5 neighbors
# knn_classifier.fit(X_train, y_train)  # Train the classifier on the training data
# y_pred = knn_classifier.predict(X_test) # Make predictions on the test data

# #############################
# KNN original - Sklearn
# knn = KNeighborsClassifier(n_neighbors=3)
# knn.fit(X_train, y_train)
# y_pred = knn.predict(X_test)

#############################
# Initialize and train the KNN classifier

# --- Configurações e Constantes ---
SEED = 78645
VALIDATE_INPUT_GEOMETRY = True
CURVATURE = 0.1  # Curvatura para o espaço de Poincaré
N_NEIGHBORS = 3  # Número de vizinhos para o KNN
TEST_SIZE_INITIAL_SPLIT = 0.1  # Proporção para o conjunto de teste
TEST_SIZE_VALIDATION_SPLIT = 0.1 # Proporção para o conjunto de validação (do restante)


# Define a semente para reprodutibilidade
np.random.seed(SEED)
torch.manual_seed(SEED)

# --- 1. Carregamento e Divisão dos Dados ---

df_data = pd.read_csv('./padufes_fair_adele_fitz_mod.csv', delimiter=',')

data = df_data.iloc[:, 1:]
data_labels = df_data.iloc[:, 0]

# Divide o dataset em treino+validação e teste
x_to_train_valid, X_test_np, y_to_train_valid, y_test_np = train_test_split(
    data,
    data_labels,
    stratify=data_labels,
    test_size=TEST_SIZE_INITIAL_SPLIT,
    random_state=SEED,
)

# Divide o conjunto treino+validação em treino e validação
X_train_np, X_valid_np, y_train_np, y_valid_np = train_test_split(
    x_to_train_valid,
    y_to_train_valid,
    stratify=y_to_train_valid,
    test_size=TEST_SIZE_VALIDATION_SPLIT,
    random_state=SEED,
)

scaler = StandardScaler()
X_train_np = scaler.fit_transform(X_train_np)
X_valid_np = scaler.transform(X_valid_np)
X_test_np = scaler.transform(X_test_np)

# --- 2. Conversão para Tensores PyTorch ---
X_train = torch.Tensor(X_train_np)
y_train = torch.Tensor(y_train_np.values).long()  # Rótulos como LongTensor
X_valid = torch.Tensor(X_valid_np)
y_valid = torch.Tensor(y_valid_np.values).long()  # Rótulos como LongTensor
X_test = torch.Tensor(X_test_np)
y_test = torch.Tensor(y_test_np.values).long()    # Rótulos como LongTensor

# --- 3. Transformação para o Espaço de Poincaré e Treinamento do Modelo ---
print(f"\nIniciando modelagem em Espaço Hiperbólico (Curvatura K={CURVATURE})...")

# Inicializa o transformador Euclidiano para Poincaré
e2p_transformer = ToPoincare(c=CURVATURE, train_c=False, train_x=False)

# Mapeia os dados para a bola de Poincaré
X_train_hyp = e2p_transformer(X_train)
X_valid_hyp = e2p_transformer(X_valid)
X_test_hyp = e2p_transformer(X_test)

# Inicializa e treina o classificador KNN Hiperbólico
hyperbolic_knn_classifier = HyperbolicKNN(k=N_NEIGHBORS, c=CURVATURE)
hyperbolic_knn_classifier.fit(X_train_hyp, y_train)

# Faz predições no conjunto de teste
y_pred_eval = hyperbolic_knn_classifier.predict(X_test_hyp)

# --- 4. Avaliação do Modelo ---

# Converte tensores para arrays NumPy para uso com sklearn.metrics
y_test_eval = y_test.cpu().numpy()

accuracy = accuracy_score(y_test_eval, y_pred_eval)
balanced_accuracy = balanced_accuracy_score(y_test_eval, y_pred_eval)
# average='weighted' para lidar com desbalanceamento de classes nas métricas de recall, precision, f1
recall = recall_score(y_test_eval, y_pred_eval, average='weighted', zero_division=0)
precision = precision_score(y_test_eval, y_pred_eval, average='weighted', zero_division=0)
f1 = f1_score(y_test_eval, y_pred_eval, average='weighted', zero_division=0)

print("\n--- Resultados da Avaliação ---")
print(f"Acurácia: {accuracy:.4f}")
print(f"Acurácia Balanceada: {balanced_accuracy:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Precisão: {precision:.4f}")
print(f"F1-Score: {f1:.4f}")


Iniciando modelagem em Espaço Hiperbólico (Curvatura K=0.1)...

--- Resultados da Avaliação ---
Acurácia: 0.8609
Acurácia Balanceada: 0.8614
Recall: 0.8609
Precisão: 0.8616
F1-Score: 0.8610


In [90]:
# --- Configurações e Constantes ---
SEED = 78645
VALIDATE_INPUT_GEOMETRY = True
CURVATURE = 0.1  # Curvatura para o espaço de Poincaré
N_NEIGHBORS = 3  # Número de vizinhos para o KNN
TEST_SIZE_INITIAL_SPLIT = 0.1  # Proporção para o conjunto de teste
TEST_SIZE_VALIDATION_SPLIT = 0.1 # Proporção para o conjunto de validação (do restante)


# Define a semente para reprodutibilidade
np.random.seed(SEED)
torch.manual_seed(SEED)

# --- 1. Carregamento e Divisão dos Dados ---

df_data = pd.read_csv('./padufes_fair_adele_fitz_mod.csv', delimiter=',')

data = df_data.iloc[:, 1:]
data_labels = df_data.iloc[:, 0]

# Divide o dataset em treino+validação e teste
x_to_train_valid, X_test_np, y_to_train_valid, y_test_np = train_test_split(
    data,
    data_labels,
    stratify=data_labels,
    test_size=TEST_SIZE_INITIAL_SPLIT,
    random_state=SEED,
)

# Divide o conjunto treino+validação em treino e validação
X_train_np, X_valid_np, y_train_np, y_valid_np = train_test_split(
    x_to_train_valid,
    y_to_train_valid,
    stratify=y_to_train_valid,
    test_size=TEST_SIZE_VALIDATION_SPLIT,
    random_state=SEED,
)

scaler = StandardScaler()
X_train_np = scaler.fit_transform(X_train_np)
X_valid_np = scaler.transform(X_valid_np)
X_test_np = scaler.transform(X_test_np)

# --- 2. Conversão para Tensores PyTorch ---
X_train = torch.Tensor(X_train_np)
y_train = torch.Tensor(y_train_np.values).long()  # Rótulos como LongTensor
X_valid = torch.Tensor(X_valid_np)
y_valid = torch.Tensor(y_valid_np.values).long()  # Rótulos como LongTensor
X_test = torch.Tensor(X_test_np)
y_test = torch.Tensor(y_test_np.values).long()    # Rótulos como LongTensor

# --- 3. Transformação para o Espaço de Poincaré e Treinamento do Modelo ---
print(f"\nIniciando KNN - SKLEARN...")

# #############################
# KNN original - Sklearn
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)
y_pred_eval = knn.predict(X_test)

# --- 4. Avaliação do Modelo ---

# Converte tensores para arrays NumPy para uso com sklearn.metrics
y_test_eval = y_test.cpu().numpy()

accuracy = accuracy_score(y_test_eval, y_pred_eval)
balanced_accuracy = balanced_accuracy_score(y_test_eval, y_pred_eval)
# average='weighted' para lidar com desbalanceamento de classes nas métricas de recall, precision, f1
recall = recall_score(y_test_eval, y_pred_eval, average='weighted', zero_division=0)
precision = precision_score(y_test_eval, y_pred_eval, average='weighted', zero_division=0)
f1 = f1_score(y_test_eval, y_pred_eval, average='weighted', zero_division=0)

print("\n--- Resultados da Avaliação ---")
print(f"Acurácia: {accuracy:.4f}")
print(f"Acurácia Balanceada: {balanced_accuracy:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Precisão: {precision:.4f}")
print(f"F1-Score: {f1:.4f}")


Iniciando KNN - SKLEARN...

--- Resultados da Avaliação ---
Acurácia: 0.8522
Acurácia Balanceada: 0.8531
Recall: 0.8522
Precisão: 0.8535
F1-Score: 0.8523


/home/labcin/anaconda3/envs/mbrenv/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [94]:
X_train = np.array(X_train)
y_train = np.array(y_train)
X_valid = np.array(X_valid)
y_valid = np.array(y_valid)
X_test = np.array(X_test)
y_test = np.array(y_test)

# --- Configurações e Constantes ---
SEED = 78645
VALIDATE_INPUT_GEOMETRY = True
CURVATURE = 0.1  # Curvatura para o espaço de Poincaré
N_NEIGHBORS = 3  # Número de vizinhos para o KNN
TEST_SIZE_INITIAL_SPLIT = 0.1  # Proporção para o conjunto de teste
TEST_SIZE_VALIDATION_SPLIT = 0.1 # Proporção para o conjunto de validação (do restante)


# Define a semente para reprodutibilidade
np.random.seed(SEED)
torch.manual_seed(SEED)

# --- 1. Carregamento e Divisão dos Dados ---

df_data = pd.read_csv('./padufes_fair_adele_fitz_mod.csv', delimiter=',')

data = df_data.iloc[:, 1:]
data_labels = df_data.iloc[:, 0]

# Divide o dataset em treino+validação e teste
x_to_train_valid, X_test_np, y_to_train_valid, y_test_np = train_test_split(
    data,
    data_labels,
    stratify=data_labels,
    test_size=TEST_SIZE_INITIAL_SPLIT,
    random_state=SEED,
)

# Divide o conjunto treino+validação em treino e validação
X_train_np, X_valid_np, y_train_np, y_valid_np = train_test_split(
    x_to_train_valid,
    y_to_train_valid,
    stratify=y_to_train_valid,
    test_size=TEST_SIZE_VALIDATION_SPLIT,
    random_state=SEED,
)

scaler = StandardScaler()
X_train_np = scaler.fit_transform(X_train_np)
X_valid_np = scaler.transform(X_valid_np)
X_test_np = scaler.transform(X_test_np)

# --- 2. Conversão para Tensores PyTorch ---
# X_train = torch.Tensor(X_train_np)
# y_train = torch.Tensor(y_train_np.values).long()  # Rótulos como LongTensor
# X_valid = torch.Tensor(X_valid_np)
# y_valid = torch.Tensor(y_valid_np.values).long()  # Rótulos como LongTensor
# X_test = torch.Tensor(X_test_np)
# y_test = torch.Tensor(y_test_np.values).long()    # Rótulos como LongTensor

# --- 3. Transformação para o Espaço de Poincaré e Treinamento do Modelo ---
print(f"\nIniciando KNN - Implementado...")

# #############################
# KNN original - Sklearn
knn_classifier = KNN(k=3)  # Create a KNN classifier with k=5 neighbors
knn_classifier.fit(X_train, y_train)  # Train the classifier on the training data
y_pred_eval = knn_classifier.predict(X_test) # Make predictions on the test data


# --- 4. Avaliação do Modelo ---

# Converte tensores para arrays NumPy para uso com sklearn.metrics
y_test_eval = y_test

accuracy = accuracy_score(y_test_eval, y_pred_eval)
balanced_accuracy = balanced_accuracy_score(y_test_eval, y_pred_eval)
# average='weighted' para lidar com desbalanceamento de classes nas métricas de recall, precision, f1
recall = recall_score(y_test_eval, y_pred_eval, average='weighted', zero_division=0)
precision = precision_score(y_test_eval, y_pred_eval, average='weighted', zero_division=0)
f1 = f1_score(y_test_eval, y_pred_eval, average='weighted', zero_division=0)

print("\n--- Resultados da Avaliação ---")
print(f"Acurácia: {accuracy:.4f}")
print(f"Acurácia Balanceada: {balanced_accuracy:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Precisão: {precision:.4f}")
print(f"F1-Score: {f1:.4f}")


Iniciando KNN - Implementado...

--- Resultados da Avaliação ---
Acurácia: 0.8522
Acurácia Balanceada: 0.8531
Recall: 0.8522
Precisão: 0.8535
F1-Score: 0.8523
